Finding phages for a given host through the Phagescope database, and possibly in close relatives if the host is missing in the list of bacteria

In [1]:
!pip install fuzzywuzzy
from google.colab import drive
import pandas as pd
from fuzzywuzzy import process
drive.mount('/content/drive')

# Load the integrated phage scope data file
combined_data_path = '/content/drive/MyDrive/integrated_phagescope_data.csv'
combined_data = pd.read_csv(combined_data_path)

# Check if the 'Host' column exists
if 'Host' not in combined_data.columns:
    raise ValueError("The 'Host' column is not found in the integrated data.")

# Split the 'Host' column by commas or semicolons, and create a single series of hosts
hosts_expanded = combined_data['Host'].str.split(',|;', expand=True).stack().reset_index(drop=True)

# Strip whitespace and filter out any empty entries
hosts_expanded = hosts_expanded.str.strip()
hosts_expanded = hosts_expanded[hosts_expanded != '']  # Remove empty strings

# Get unique hosts
unique_hosts = pd.Series(hosts_expanded).unique()
unique_hosts_list = sorted(unique_hosts)

# Function to search for a host with fuzzy matching
def search_host(target_host):
    match = process.extractOne(target_host, unique_hosts_list)
    return match

# Ask user for the target host name
target_host = input("Enter the target host name: ")

# Search for the target host in the unique hosts list
result = search_host(target_host)

# Check if a match was found
if result and result[1] >= 80:  # Confidence threshold
    print(f"Match found: {result[0]} (Confidence: {result[1]}%)")

    # Retrieve all phage IDs matching the found host
    matching_host = result[0]
    # Filter the original dataframe for the matching host
    combined_data_cleaned = combined_data.dropna(subset=['Host'])  # Drop rows where 'Host' is NaN
    matching_phages = combined_data_cleaned[combined_data_cleaned['Host'].str.contains(matching_host, case=False)]

    # Extract the Phage_IDs
    phage_ids = matching_phages['Phage_ID'].tolist()

    if phage_ids:
        print("Phage IDs matching the host:")
        for phage_id in phage_ids:
            print(phage_id)
    else:
        print("No phage IDs found for this host.")
else:
    print("No close match found for the specified host. Proceeding with search of relatives in the host list.")
    !pip install biopython
    import requests
    from Bio import Entrez
    from fuzzywuzzy import process
    # Function to get taxonomy details
    def get_taxonomy_info(taxid):
        if not isinstance(taxid, str) or not taxid.isdigit():
            print(f"Invalid taxid: {taxid}")
            return None
        try:
            fetch_handle = Entrez.efetch(db="taxonomy", id=taxid, retmode="xml")
            fetch_results = Entrez.read(fetch_handle)
            fetch_handle.close()

            if not fetch_results:
                print(f"No taxonomy information found for TaxID: {taxid}")
                return None

            lineage = fetch_results[0].get('Lineage', '').split("; ")
            scientific_name = fetch_results[0].get('ScientificName', None)

            genus = lineage[-2] if len(lineage) >= 2 else None
            family = lineage[-3] if len(lineage) >= 3 else None

            return {
                "lineage": lineage,
                "genus": genus,
                "family": family,
                "scientific_name": scientific_name
            }

        except Exception as e:
            print(f"Error fetching taxonomy information for TaxID: {taxid}, Error: {e}")
            return None

    # Function to find species in the same genus or family using NCBI Entrez
    def find_species_in_taxonomy(taxid, rank):
        search_handle = Entrez.esearch(db="taxonomy", term=f"txid{taxid}[Subtree] AND {rank}[Rank]", retmax=100)
        search_results = Entrez.read(search_handle)
        search_handle.close()

        if not search_results["IdList"]:
            print(f"No {rank}-level relatives found for TaxID: {taxid}")
            return []

        species_list = []
        for species_id in search_results["IdList"]:
            species_info = get_taxonomy_info(species_id)
            if species_info and "scientific_name" in species_info and species_info["scientific_name"]:
                species_list.append(species_info["scientific_name"])

        return species_list

    def get_taxid(species_name):
        try:
            Entrez.email = "your_email@example.com"  # Always provide your email for NCBI
            search_handle = Entrez.esearch(db="taxonomy", term=species_name)
            search_results = Entrez.read(search_handle)
            search_handle.close()
            if search_results["IdList"]:
                return search_results["IdList"][0]  # Return the first match's taxid
            else:
                print(f"No TaxID found for species: {species_name}")
                return None
        except Exception as e:
            print(f"Error retrieving TaxID for {species_name}: {e}")
            return None

    # Function to find the closest relatives based on taxonomy information
    def find_closest_relatives(taxid, host_list):
        taxonomy_info = get_taxonomy_info(taxid)
        if taxonomy_info is None:
            print(f"No taxonomy information available for TaxID {taxid}.")
            return []

        genus_species = find_species_in_taxonomy(taxid, 'species')
        matches = find_best_fuzzy_matches(genus_species, host_list, confidence_threshold=94)

        if matches:
            return matches

        family_species = []
        if taxonomy_info['family']:
            family_taxid = get_taxid(taxonomy_info['family'])  # Assuming get_taxid is defined elsewhere
            if family_taxid:
                family_species = find_species_in_taxonomy(family_taxid, 'species')

        return find_best_fuzzy_matches(family_species, host_list, confidence_threshold=94)

    # Function to find the best fuzzy matches with a confidence threshold
    def find_best_fuzzy_matches(species_list, host_list, confidence_threshold):
        matches_dict = {}

        for species_name in species_list:
            best_match = process.extractOne(species_name, host_list)
            if best_match and best_match[1] >= confidence_threshold:
                match_name = best_match[0]
                if match_name not in matches_dict:
                    matches_dict[match_name] = []
                matches_dict[match_name].append(species_name)

        # Prepare the output list
        output = []
        for match, species_names in matches_dict.items():
            species_group = ", ".join(species_names)
            output.append(f"Species: {species_group} | Best Match: {match}")

        return output

    # Example usage
    taxid = input("Enter the target species Taxonomy ID on NCBI Taxonomy browser: ")
    matching_relatives = find_closest_relatives(taxid, unique_hosts_list)

    print("Matching relatives in the host list:")
    for relative in matching_relatives:
        print(relative)
    target_host = input("Try the phage search entering one of the relatives name:")
    result = search_host(target_host)
    # Check if a match was found
    if result and result[1] >= 80:  # Confidence threshold
        print(f"Match found: {result[0]} (Confidence: {result[1]}%)")

        # Retrieve all phage IDs matching the found host
        matching_host = result[0]
        # Filter the original dataframe for the matching host
        combined_data_cleaned = combined_data.dropna(subset=['Host'])  # Drop rows where 'Host' is NaN
        matching_phages = combined_data_cleaned[combined_data_cleaned['Host'].str.contains(matching_host, case=False)]

        # Extract the Phage_IDs
        phage_ids = matching_phages['Phage_ID'].tolist()

        if phage_ids:
            print("Phage IDs matching the host:")
            for phage_id in phage_ids:
                print(phage_id)
        else:
            print("No phage IDs found for this host.")
    else:
        print("No close match found for the specified host. Proceeding with search of relatives in the host list.")



/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-1-e33bd778c136>:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_data = pd.read_csv(combined_data_path)


Enter the target host name: synechococcus
Match found: Synechococcus (Confidence: 100%)
Phage IDs matching the host:
NC_006820.1
NC_009531.1
NC_008296.2
NC_013085.1
NC_015279.1
NC_015281.1
NC_015282.1
NC_015285.1
NC_015286.1
NC_015287.1
NC_015288.1
NC_015289.1
NC_015463.1
NC_015465.1
NC_015569.1
NC_016164.1
NC_016656.1
NC_016766.1
NC_019443.1
NC_019444.1
NC_019516.1
NC_020486.1
NC_020837.1
NC_020838.1
NC_020851.1
NC_020854.1
NC_020859.1
NC_020865.1
NC_020867.1
NC_020875.1
NC_021072.1
NC_021530.1
NC_021536.1
NC_023584.1
NC_023587.1
NC_025455.1
NC_025456.1
NC_025461.1
NC_025464.1
NC_026926.1
NC_026923.1
NC_026924.1
NC_026927.1
NC_026928.1
NC_027132.1
NC_027130.1
NC_003390.2
NC_029031.1
NC_031242.1
NC_031900.1
NC_031903.1
NC_031906.1
NC_031922.1
NC_031927.1
NC_031935.1
NC_031944.1
NC_047712.1
NC_047713.1
NC_047714.1
NC_047715.1
NC_047716.1
NC_047717.1
NC_047718.1
NC_047719.1
NC_047733.1
NC_047734.1
NC_047838.1
NC_048015.1
NC_048026.1
NC_048049.1
NC_048102.1
NC_048106.1
NC_048171.1
NC_0481

Once you found the phages, get the proteins.

In [2]:
filtered_df = pd.read_csv('/content/drive/MyDrive/filtered_phage_data.tsv', sep='\t')

# Filter the DataFrame to get only the rows where the Phage_ID matches one of the phage_ids
matching_proteins = filtered_df[filtered_df['Phage_ID'].isin(phage_ids)]

# Group the matching proteins by 'Phage_ID'
grouped_proteins = matching_proteins.groupby('Phage_ID')

# Create an output dictionary to store proteins grouped by Phage_ID
output_dict = {phage_id: group[['Protein_ID', 'Product']].to_dict('records') for phage_id, group in grouped_proteins}

# Print the grouped proteins
for phage_id, proteins in output_dict.items():
    print(f"Phage ID: {phage_id}")
    for protein in proteins:
        print(f"  Protein ID: {protein['Protein_ID']}, Product: {protein['Product']}")
    print("\n")

Output streaming troncato alle ultime 5000 righe.
  Protein ID: AIX14913.1, Product: proximal tail sheath protein
  Protein ID: AIX14926.1, Product: tail sheath monomer protein
  Protein ID: AIX14927.1, Product: tail tube monomer protein
  Protein ID: AIX14928.1, Product: portal vertex protein
  Protein ID: AIX14932.1, Product: head vertex precursor
  Protein ID: AIX14934.1, Product: head-proximal tip of tail protein


Phage ID: KJ019030.1
  Protein ID: AIX15052.1, Product: base plate wedge component
  Protein ID: AIX15053.1, Product: baseplate tail tube cap protein
  Protein ID: AIX15056.1, Product: baseplate hub subunit
  Protein ID: AIX15057.1, Product: base plate hub assembly catalyst
  Protein ID: AIX15066.1, Product: baseplate hub + tail lysozyme
  Protein ID: AIX15078.1, Product: putative short tail fiber
  Protein ID: AIX15125.1, Product: base plate wedge subunit
  Protein ID: AIX15126.1, Product: baseplate wedge protein
  Protein ID: AIX15129.1, Product: baseplate wedge tail f

get the protein sequences for the found proteins

In [3]:
import pandas as pd

# Load the CSV file (replace 'your_file.csv' with the actual path to your CSV file)
csv_file_path = '/content/drive/MyDrive/filtered_merged_output.csv'
df = pd.read_csv(csv_file_path, header=None)

# Extract the list of Protein_IDs from output_dict
protein_ids = {entry['Protein_ID'] for key in output_dict for entry in output_dict[key]}

# Filter the DataFrame to keep only rows where the first column matches one of the Protein_IDs
filtered_df = df[df[0].isin(protein_ids)]

# Extract the second column as a list
second_column_values = filtered_df[1].tolist()

# The result is stored in `second_column_values`
second_column_values

<ipython-input-3-ae845f562ebf>:5: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path, header=None)


['...',
 'MTEIENLIPWEREVYTSLLIQHIKEEKQKAEEAKSRR',
 '...',
 'MPWEKEIYVNMLIQFLKEEEKRMKEQQAASG',
 '...',
 ...]

now I want a script that, given a GenBank accession number for the genome of the target, connects to PHASTEST to get the sequences of the phage genes

In [4]:
import requests
import time

# Prompt user for accession number
accession_number = input("Enter the GenBank accession number: ")

# URL to access PHASTEST job details
url = f"https://phastest.ca/jobs/{accession_number}/detail.txt"

try:
    # Send GET request to the URL
    response = requests.get(url)

    # Wait for a short time if the server needs a delay
    time.sleep(3)  # Adjust sleep time if necessary

    # Check if the request was successful
    if response.status_code == 200:
        # Get the text response
        data = response.text

        # Keywords to search for
        keywords = [
            "tail protein",
            "phage-like protein",
            "portal protein",
            "hypothetical protein",
            "head protein",
            "plate protein",
            "fiber protein",
            "membrane protein"
        ]

        # List to store protein sequences
        protein_sequences = []

        # Extract lines containing the keywords
        for line in data.splitlines():
            if any(keyword in line.lower() for keyword in keywords):
                print(line)

                # Find and extract the protein sequence (assuming it's at the end of each line)
                sequence = line.split()[-1]

                # Add sequence to protein_sequences list
                protein_sequences.append(sequence)

        # Print all collected sequences for verification
        print("\nCollected Protein Sequences:")
        for seq in protein_sequences:
            print(seq)

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

except Exception as e:
    print(f"An error occurred: {e}")


Enter the GenBank accession number: NZ_AP014565.1
complement(370630..370755)         PHAGE_Salmon_118970_sal3_NC_031940: hypothetical protein; PP_00326; phage            1.00e-20            -                   NONE                               NONE                               NONE                               MTTITKERIELYVKSPLENGLTRGEQMDLARIALASLEAEPI
complement(371683..371982)         PHAGE_Escher_HK75_NC_016160: hypothetical protein; PP_00329; phage                   2.98e-65            -                   NONE                               NONE                               NONE                               MSVCLIDKRRRGQQIPSVEMPNHTWFCVLDIDGMDTLIDTRHYCDTATATPAKAKKMAALIENWTPPDGWCNGNDRDWHEKMKGYICDFLRKCNGFMVM
complement(371979..372377)         PHAGE_Escher_ECP1_NC_049926: hypothetical protein; PP_00330; phage                   2.58e-39            -                   NONE                               NONE                               NONE                               MKQMTLIEMDGF

Look for identities between the infection proteins of the prophages and those of the phages of close relatives of the target

In [ ]:
!pip install biopython
from Bio import pairwise2
from Bio.Seq import Seq
import concurrent.futures

# Set parameters
similarity_threshold = 0.7
length_difference_threshold = 0.2  # Allowable length difference ratio (e.g., 20%)

# Store significant homologies
significant_homologies = []

# Function to calculate similarity and filter results
def calculate_similarity(seq_id, input_seq, output_seq):
    # Skip pairs with length difference beyond threshold
    if abs(len(input_seq) - len(output_seq)) / max(len(input_seq), len(output_seq)) > length_difference_threshold:
        return None

    # Perform local alignment and get only the best alignment
    alignments = pairwise2.align.localxx(input_seq, output_seq, one_alignment_only=True)

    # Calculate similarity for the best alignment
    for alignment in alignments:
        similarity = alignment[2] / alignment[4]
        if similarity >= similarity_threshold:
            return {
                'Input_Sequence_ID': seq_id,
                'Output_Protein_ID': "N/A",
                'Output_Product': "N/A",
                'Similarity': similarity,
                'Alignment': alignment
            }
    return None

# Parallelize the alignment process
def find_significant_homologies(protein_sequences, second_column_values):
    results = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Submit each pair of sequences for parallel processing
        futures = [
            executor.submit(calculate_similarity, seq_id, input_seq, Seq(output_seq))
            for seq_id, input_seq in enumerate(protein_sequences)
            for output_seq in second_column_values
        ]

        # Collect results as they complete
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                results.append(result)
                print(f"Significant homology found for Input Sequence ID {result['Input_Sequence_ID']} with similarity {result['Similarity']:.2f}")

    return results

# Run the homology search and display results
significant_homologies = find_significant_homologies(protein_sequences, second_column_values)

# Display results
for homology in significant_homologies:
    print(f"Significant homology found:\n"
          f"Input Sequence ID: {homology['Input_Sequence_ID']}\n"
          f"Output Protein ID: {homology['Output_Protein_ID']}\n"
          f"Product: {homology['Output_Product']}\n"
          f"Similarity: {homology['Similarity']:.2f}\n"
          f"Alignment:\n{homology['Alignment']}\n")
